In [ ]:
import os
import json
import pickle
import mir_eval
import numpy as np

import sys
sys.path.append("..")
from utils.event import expand_score
from utils.align import get_t_bar, get_t_fin
from utils.common import get_file_list, load_event
from utils.constants import DATA_DIR

event_dir = os.path.join(DATA_DIR, "event")
midi_dir = os.path.join(DATA_DIR, "midi")
true_dir = os.path.join(DATA_DIR, "struct_annotated")
pred_dir = os.path.join(DATA_DIR, "struct_mark")
pred_theme_dir = os.path.join(DATA_DIR, "struct_mark_theme")
triplet_bound_dir = os.path.join(DATA_DIR, "boundary_predictions")

def get_t_bounds(struct):
    sect_bounds = []
    theme_bounds = []
    for sect in struct.values():
        sect_bounds.append(sect[-1]['end t'])
        for theme in sect:
            theme_bounds.append(theme['end t'])
    return sect_bounds, theme_bounds

def get_ts_cpt(composer, basename):

    event_file = os.path.join(event_dir, composer, basename)
    score_event, mark = load_event(event_file)
    event, _ = expand_score(score_event, mark)

    map_file = os.path.join(midi_dir, composer, basename)
    with open(map_file) as f:
        ts_cpt = json.load(f)["cpt"]
    t_fin = get_t_fin(max(event), ts_cpt)
    ts_cpt.append({"measure": max(event) + 1, 't': float(t_fin),
                   "tempo": ts_cpt[-1]["tempo"],
                   "time_signature": ts_cpt[-1]["time_signature"]})
    return ts_cpt

def bound_to_interval(bounds):
    bounds = [0] + bounds
    return np.array([bounds[:-1], bounds[1:]]).T

In [13]:
struct_file_list = get_file_list(true_dir)

sect_p5_res, sect_3_res, sect_m_res = [], [], []
theme_p5_res, theme_3_res, theme_m_res = [], [], []
phrase_p5_res, phrase_3_res, phrase_m_res = [], [], []

# Baseline
b_sect_p5_res, b_sect_3_res, b_sect_m_res = [], [], []
b_theme_p5_res, b_theme_3_res, b_theme_m_res = [], [], []
b_phrase_p5_res, b_phrase_3_res, b_phrase_m_res = [], [], []

for struct_file in struct_file_list:
    
    composer, basename = struct_file.split(".json")[0].split("/")[-2:]
    
    ts_cpt = get_ts_cpt(composer, f"{basename}.json")
    
    max_t_bar = max([get_t_bar(**i) for i in ts_cpt])
    
    with open(struct_file) as f:
        struct = json.load(f)['struct']
    
    sect_bounds, theme_bounds = get_t_bounds(struct)
     
    pred_struct_file = os.path.join(pred_dir, f"{composer}-{basename}.json")
    with open(pred_struct_file) as f:
        pred_struct = json.load(f)
    pred_sect_bounds, pred_phrase_bounds = get_t_bounds(pred_struct)

    pred_theme_file = os.path.join(pred_theme_dir, f"{composer}-{basename}.json")
    with open(pred_theme_file) as f:
        pred_theme = json.load(f)
    _, pred_theme_bounds = get_t_bounds(pred_theme)

    sect_intervals = bound_to_interval(sect_bounds)
    pred_sect_intervals = bound_to_interval(pred_sect_bounds)
    sect_p5_res += [mir_eval.segment.detection(sect_intervals, pred_sect_intervals, window=.5, )]
    sect_3_res += [mir_eval.segment.detection(sect_intervals, pred_sect_intervals, window=3, )]
    sect_m_res += [mir_eval.segment.detection(sect_intervals, pred_sect_intervals, window=max_t_bar, )]

    theme_intervals = bound_to_interval(theme_bounds)
    pred_theme_intervals = bound_to_interval(pred_theme_bounds)
    theme_p5_res += [mir_eval.segment.detection(theme_intervals, pred_theme_intervals, window=.5, )]
    theme_3_res += [mir_eval.segment.detection(theme_intervals, pred_theme_intervals, window=3, )]
    theme_m_res += [mir_eval.segment.detection(theme_intervals, pred_theme_intervals, window=max_t_bar, )]

    pred_phrase_intervals = bound_to_interval(pred_phrase_bounds)
    phrase_p5_res += [mir_eval.segment.detection(theme_intervals, pred_phrase_intervals, window=.5, )]
    phrase_3_res += [mir_eval.segment.detection(theme_intervals, pred_phrase_intervals, window=3, )]
    phrase_m_res += [mir_eval.segment.detection(theme_intervals, pred_phrase_intervals, window=max_t_bar, )]

    # Baseline
    pred_bound_file = os.path.join(triplet_bound_dir, f"{composer}-{basename}.pkl")
    with open(pred_bound_file, "rb") as f:
        pred_bounds = pickle.load(f)

    n_sect_level = 3
    sect_intervals = pred_bounds[n_sect_level][0]
    pred_sect_intervals = bound_to_interval(pred_sect_bounds)
    b_sect_p5_res += [mir_eval.segment.detection(sect_intervals, pred_sect_intervals, window=.5, )]
    b_sect_3_res += [mir_eval.segment.detection(sect_intervals, pred_sect_intervals, window=3, )]
    b_sect_m_res += [mir_eval.segment.detection(sect_intervals, pred_sect_intervals, window=max_t_bar, )]

    n_theme_level = 6
    theme_intervals = pred_bounds[n_theme_level][0]
    pred_theme_intervals = bound_to_interval(pred_theme_bounds)
    b_theme_p5_res += [mir_eval.segment.detection(theme_intervals, pred_theme_intervals, window=.5, )]
    b_theme_3_res += [mir_eval.segment.detection(theme_intervals, pred_theme_intervals, window=3, )]
    b_theme_m_res += [mir_eval.segment.detection(theme_intervals, pred_theme_intervals, window=max_t_bar, )]

    n_phrase_level = -1
    pred_phrase_intervals = pred_bounds[n_phrase_level][0]
    b_phrase_p5_res += [mir_eval.segment.detection(theme_intervals, pred_phrase_intervals, window=.5, )]
    b_phrase_3_res += [mir_eval.segment.detection(theme_intervals, pred_phrase_intervals, window=3, )]
    b_phrase_m_res += [mir_eval.segment.detection(theme_intervals, pred_phrase_intervals, window=max_t_bar, )]

In [14]:
# precision, recall, f1
avg_sect_p5 = np.mean(np.array(sect_p5_res), axis=0)
avg_sect_3 = np.mean(np.array(sect_3_res), axis=0)
avg_sect_m = np.mean(np.array(sect_m_res), axis=0)

avg_theme_p5 = np.mean(np.array(theme_p5_res), axis=0)
avg_theme_3 = np.mean(np.array(theme_3_res), axis=0)
avg_theme_m = np.mean(np.array(theme_m_res), axis=0)

avg_phrase_p5 = np.mean(np.array(phrase_p5_res), axis=0)
avg_phrase_3 = np.mean(np.array(phrase_3_res), axis=0)
avg_phrase_m = np.mean(np.array(phrase_m_res), axis=0)

# Baseline
avg_b_sect_p5 = np.mean(np.array(b_sect_p5_res), axis=0)
avg_b_sect_3 = np.mean(np.array(b_sect_3_res), axis=0)
avg_b_sect_m = np.mean(np.array(b_sect_m_res), axis=0)

avg_b_theme_p5 = np.mean(np.array(b_theme_p5_res), axis=0)
avg_b_theme_3 = np.mean(np.array(b_theme_3_res), axis=0)
avg_b_theme_m = np.mean(np.array(b_theme_m_res), axis=0)

avg_b_phrase_p5 = np.mean(np.array(b_phrase_p5_res), axis=0)
avg_b_phrase_3 = np.mean(np.array(b_phrase_3_res), axis=0)
avg_b_phrase_m = np.mean(np.array(b_phrase_m_res), axis=0)

print('Section Boundary')
print(f"{'Method 1':<60} {'Baseline'}")
print(f"{'Window':<12} {'Precision':<12} {'Recall':<12} {'F1':<21} {'Precision':<12} {'Recall':<12} {'F1'}")
print(f"{'.5':<12} {avg_sect_p5[0]:<12.4f} {avg_sect_p5[1]:<12.4f} {avg_sect_p5[2]:<21.4f} {avg_b_sect_p5[0]:<12.4f} {avg_b_sect_p5[1]:<12.4f} {avg_b_sect_p5[2]:<12.4f}")
print(f"{'3':<12} {avg_sect_3[0]:<12.4f} {avg_sect_3[1]:<12.4f} {avg_sect_3[2]:<21.4f} {avg_b_sect_3[0]:<12.4f} {avg_b_sect_3[1]:<12.4f} {avg_b_sect_3[2]:<12.4f}")
print(f"{'measure':<12} {avg_sect_m[0]:<12.4f} {avg_sect_m[1]:<12.4f} {avg_sect_m[2]:<21.4f} {avg_b_sect_m[0]:<12.4f} {avg_b_sect_m[1]:<12.4f} {avg_b_sect_m[2]:<12.4f}")
print()

print('Thematic Material Boundary')
print(f"{'Method 1':<60} {'Baseline'}")
print(f"{'Window':<12} {'Precision':<12} {'Recall':<12} {'F1':<21} {'Window':<12}")
print(f"{'.5':<12} {avg_theme_p5[0]:<12.4f} {avg_theme_p5[1]:<12.4f} {avg_theme_p5[2]:<21.4f} {avg_b_theme_p5 [0]:<12.4f} {avg_b_theme_p5 [1]:<12.4f} {avg_b_theme_p5 [2]:<12.4f}")
print(f"{'3':<12} {avg_theme_3[0]:<12.4f} {avg_theme_3[1]:<12.4f} {avg_theme_3[2]:<21.4f} {avg_b_theme_3[0]:<12.4f} {avg_b_theme_3[1]:<12.4f} {avg_b_theme_3[2]:<12.4f}")
print(f"{'measure':<12} {avg_theme_m[0]:<12.4f} {avg_theme_m[1]:<12.4f} {avg_theme_m[2]:<21.4f} {avg_b_theme_m[0]:<12.4f} {avg_b_theme_m[1]:<12.4f} {avg_b_theme_m[2]:<12.4f}")
print()

print("Phrase-Level Material Boundary")
print(f"{'Method 1':<60} {'Baseline'}")
print(f"{'Window':<12} {'Precision':<12} {'Recall':<12} {'F1':<21} {'Window':<12}")
print(f"{'.5':<12} {avg_phrase_p5[0]:<12.4f} {avg_phrase_p5[1]:<12.4f} {avg_phrase_p5[2]:<21.4f} {avg_b_phrase_p5 [0]:<12.4f} {avg_b_phrase_p5 [1]:<12.4f} {avg_b_phrase_p5 [2]:<12.4f}")
print(f"{'3':<12} {avg_phrase_3[0]:<12.4f} {avg_phrase_3[1]:<12.4f} {avg_phrase_3[2]:<21.4f} {avg_b_phrase_3[0]:<12.4f} {avg_b_phrase_3[1]:<12.4f} {avg_b_phrase_3[2]:<12.4f}")
print(f"{'measure':<12} {avg_phrase_m[0]:<12.4f} {avg_phrase_m[1]:<12.4f} {avg_phrase_m[2]:<21.4f} {avg_b_phrase_m[0]:<12.4f} {avg_b_phrase_m[1]:<12.4f} {avg_b_phrase_m[2]:<12.4f}")


# print("Phrase-Level Material Boundary")
# print('Baseline')
# print(f"{'Window':<12} {'Precision':<12} {'Recall':<12} {'F1':<12}")
# print(f"{'.5':<12} {avg_b_phrase_p5[0]:<12.4f} {avg_b_phrase_p5[1]:<12.4f} {avg_b_phrase_p5[2]:<12.4f}")
# print(f"{'3':<12} {avg_b_phrase_3[0]:<12.4f} {avg_b_phrase_3[1]:<12.4f} {avg_b_phrase_3[2]:<12.4f}")
# print(f"{'measure':<12} {avg_b_phrase_m[0]:<12.4f} {avg_b_phrase_m[1]:<12.4f} {avg_b_phrase_m[2]:<12.4f}")

Section Boundary
Method 1                                                     Baseline
Window       Precision    Recall       F1                    Precision    Recall       F1
.5           0.7951       0.7928       0.7919                0.5333       0.1569       0.2385      
3            0.9083       0.9043       0.9041                0.8585       0.2525       0.3836      
measure      0.8653       0.8622       0.8616                0.7589       0.2219       0.3375      

Thematic Material Boundary
Method 1                                                     Baseline
Window       Precision    Recall       F1                    Window      
.5           0.3860       0.4701       0.4135                0.8094       0.6296       0.6925      
3            0.5987       0.7602       0.6559                0.9632       0.7285       0.8070      
measure      0.5268       0.6549       0.5713                0.9301       0.7013       0.7776      

Phrase-Level Material Boundary
Method 1           

## Eval
250211 ver ('struct_mark_prev')
1. Section Boundary 
|  | Proposed Method |  |  | Baseline |  |  |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| window | Precision | Recall | F1 | Precision | Recall | F1 |
| 0.5 s | 0.8714 | 0.7289 | 0.7763 | 0.6072 | 0.1500 | 0.2349 |
| 3 s | 0.9327 | 0.7887 | 0.8368 | 0.8878 | 0.2181 | 0.3421 |
| 1 measure | 0.9117 | 0.7682 | 0.8161 | 0.8276 | 0.2003 | 0.3150 |


2. Thematic Material Boundary
| Method 1 | Baseline |  |  | Baseline |  |  |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| Window | Precision | Recall | F1 | Window | Recall | F1 |
| .5 | 0.3398 | 0.5019 | 0.3863 | 0.3343 | 0.3092 | 0.2987 |
| 3 | 0.5131 | 0.8155 | 0.6050 | 0.7043 | 0.6745 | 0.6479 |
| measure | 0.4486 | 0.6819 | 0.5189 | 0.5977 | 0.5426 | 0.5319 |

3. Phrase-Level Material Boundary
|  | Baseline |  |  |
|:---:|:---:|:---:|---|
| Window | Precision | Recall | F1 |
| .5 | 0.3191 | 0.5190 | 0.3917 |
| 3 | 0.5715 | 0.9328 | 0.7025 |
| measure | 0.5050 | 0.8227 | 0.6204 |